# Final

This final exercise combines different parts from the previous exercises.

## Section 1

### Search Digitraffic's REST interface service (in JSON format) for active trains with the endpoint address `/live-trains` from Helsinki station with the following search parameters:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [1]:
import requests

url = "https://rata.digitraffic.fi/api/v1/live-trains"

params = {
    'arrived_trains': 10,
    'departed_trains': 10,
    'include_nonstopping': False,
    'station': 'HKI'
}

response = requests.get(url=url, params=params)
trains = response.json()
#trains

### Read the data into the Pandas dataframe without editing, so that the column headings are as follows:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version`

In [2]:
import pandas as pd

df = pd.DataFrame(trains)
df.head()

,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
0,46,2025-02-15,10,vr,IC,Long-distance,,False,False,290506478051,REGULAR,2024-10-31T06:14:29.000Z,"[{'stationShortCode': 'VS', 'stationUICCode': ..."
1,956,2025-02-15,10,vr,S,Long-distance,,False,False,290506513782,REGULAR,2024-12-12T06:09:20.000Z,"[{'stationShortCode': 'TKU', 'stationUICCode':..."
2,957,2025-02-15,10,vr,S,Long-distance,,True,False,290506547413,REGULAR,2024-12-12T06:09:20.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
3,8118,2025-02-15,10,vr,HL,Commuter,A,False,False,290506522863,REGULAR,2024-10-31T06:14:29.000Z,"[{'stationShortCode': 'LPV', 'stationUICCode':..."
4,8125,2025-02-15,10,vr,HL,Commuter,A,True,False,290506530944,REGULAR,2024-10-31T06:14:29.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


### Delete all other columns except `timeTableRows` and `trainNumber`.

In [3]:
df = df[['timeTableRows', 'trainNumber']]
df.head()

,timeTableRows,trainNumber
0,"[{'stationShortCode': 'VS', 'stationUICCode': ...",46
1,"[{'stationShortCode': 'TKU', 'stationUICCode':...",956
2,"[{'stationShortCode': 'HKI', 'stationUICCode':...",957
3,"[{'stationShortCode': 'LPV', 'stationUICCode':...",8118
4,"[{'stationShortCode': 'HKI', 'stationUICCode':...",8125


### After this, extract the values ​​of the `timeTableRows` column from each train separately (the column value for each train contains several JSON objects) and add `trainNumber`, which identifies the train data.

In [4]:
df = df.explode('timeTableRows')
df

,timeTableRows,trainNumber
0,"{'stationShortCode': 'VS', 'stationUICCode': 2...",46
0,"{'stationShortCode': 'LAI', 'stationUICCode': ...",46
0,"{'stationShortCode': 'LAI', 'stationUICCode': ...",46
0,"{'stationShortCode': 'TK', 'stationUICCode': 2...",46
0,"{'stationShortCode': 'TK', 'stationUICCode': 2...",46
...,...,...
28,"{'stationShortCode': 'ILR', 'stationUICCode': ...",64173
29,"{'stationShortCode': 'ILR', 'stationUICCode': ...",64444
29,"{'stationShortCode': 'PSL', 'stationUICCode': ...",64444
29,"{'stationShortCode': 'PSL', 'stationUICCode': ...",64444


### Then drop the following columns from the data: `causes`, `differenceInMinutes` and `commercialStop`.

In [5]:
df = pd.json_normalize(df['timeTableRows'])
df = df.drop(['causes', 'differenceInMinutes', 'commercialStop'], axis=1)
df.head()

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,actualTime,trainReady.source,trainReady.accepted,trainReady.timestamp,liveEstimateTime,estimateSource
0,VS,288,FI,DEPARTURE,True,1,False,2025-02-15T08:35:00.000Z,2025-02-15T08:35:18.000Z,KUPLA,True,2025-02-15T08:21:37.000Z,NaN,NaN
1,LAI,293,FI,ARRIVAL,False,1,False,2025-02-15T08:50:00.000Z,2025-02-15T08:49:38.000Z,NaN,NaN,NaN,NaN,NaN
2,LAI,293,FI,DEPARTURE,False,1,False,2025-02-15T08:50:00.000Z,2025-02-15T08:49:38.000Z,NaN,NaN,NaN,NaN,NaN
3,TK,294,FI,ARRIVAL,True,1,False,2025-02-15T08:55:00.000Z,NaN,NaN,NaN,NaN,2025-02-15T08:54:42.000Z,LIIKE_AUTOMATIC
4,TK,294,FI,DEPARTURE,True,1,False,2025-02-15T08:56:00.000Z,NaN,NaN,NaN,NaN,2025-02-15T08:56:00.000Z,LIIKE_AUTOMATIC


### Fill in the missing values ​​of the `actualTime` column in each row with the values of the `scheduledTime` column.

In [6]:
df['actualTime'] = df['actualTime'].fillna(df['scheduledTime'])
df['actualTime']

0       2025-02-15T08:35:18.000Z
1       2025-02-15T08:49:38.000Z
2       2025-02-15T08:49:38.000Z
3       2025-02-15T08:55:00.000Z
4       2025-02-15T08:56:00.000Z
                  ...           
1455    2025-02-15T13:04:00.000Z
1456    2025-02-15T12:39:59.000Z
1457    2025-02-15T12:45:14.000Z
1458    2025-02-15T12:45:14.000Z
1459    2025-02-15T12:55:00.000Z
Name: actualTime, Length: 1460, dtype: object

## Section 2

### Extract the cell values ​​of the `table` element of the `data.html` file using the regex syntax and connect them to the dataframe created in step 1. **Note**: only regex should be used in the task (E.g. the use of Pandas' read_html() method is prohibited)!

In [7]:
import re

with open('data.html', 'r') as file:
    html_content = file.read()

pattern = r'<td>\s*(.*?)\s*</td>'
values = re.findall(pattern, html_content)
values = [v.strip() for v in values]
values

['2019-08-06T04:48:00.000Z',
 'False',
 '',
 'FI',
 '2019-08-06T04:48:00.000Z',
 'JVS',
 '1272',
 'False',
 'DEPARTURE',
 '150',
 '2019-08-06T06:04:39.000Z',
 'False',
 '',
 'FI',
 '2019-08-06T06:04:39.000Z',
 'R702',
 '98998',
 'False',
 'ARRIVAL',
 '150',
 '2019-08-06T07:03:12.000Z',
 'False',
 '',
 'FI',
 '2019-08-06T07:03:12.000Z',
 'ILA',
 '98998',
 '9',
 'ARRIVAL',
 '8337']

## Section 3

### Make a SQLite database and insert the dataframe obtained as a result of step 2 to the database.

In [8]:
import sqlite3

conn = sqlite3.connect('trains.db')
df.to_sql('trains', conn, if_exists='replace', index=False)
conn.close()